In [60]:
import pandas as pd
import pymysql
import html5lib
import requests  # ValueError: No tables found 가 뜨는 것 방지 위해
from bs4 import BeautifulSoup as bs

In [61]:
# 전역변수 선언부
conn, cur = None, None

data1, data2, data3, data4, data5, data6, data7, data8, data9 = "", "", "", "", "", "", "", "", ""

sql = ""

# 메인코드
conn = pymysql.connect(host='localhost', user='root',
                       password='1234', db='miniproject1',  charset='utf8')

cur = conn.cursor()

cur.execute(
    "CREATE TABLE IF NOT EXISTS naverStock(종목코드 char(100),날짜 char(100), 종가 char(200), 전일비 char(100), 시가 char(100), 고가 char(100), 저가 char(100), 거래량 char(100))")

0

In [62]:
# code = input('종목코드를 적어주세요')
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}

def get_daily_list(stock_code, page_no):
    url = f"https://finance.naver.com/item/sise_day.nhn?code={stock_code}&page={page_no}"
    res = requests.get(url, headers= headers)
    soup = bs(res.text, "lxml")
    table = soup.select("table")
    table = pd.read_html(str(table))
    df_daily = table[0].dropna()
    return df_daily

In [63]:
sql = "select code from krx_Table"

cur.execute(query=sql)

result = cur.fetchall()

code = pd.DataFrame(result)


In [64]:
for i in range(0,5): # 코드명 2484개
    # print(int(code[0][i]))
    for j in range(1, 20): # 페이지 
        df = get_daily_list(str(code[0][i]), j)
        for k in range(1,14): # 페이지 내 날짜별
            try:
                row1 = str(code[0][i])
                row2 = df['날짜'][k]
                row3 = df['종가'][k]
                row4 = df['전일비'][k]
                row5 = df['시가'][k]
                row6 = df['고가'][k]
                row7 = df['저가'][k]
                row8 = df['거래량'][k]

                sql = """insert into naverStock(종목코드, 날짜, 종가, 전일비, 시가, 고가, 저가, 거래량) values(%s, %s, %s, %s, %s, %s, %s, %s);"""
                cur.execute(sql, (row1, row2, row3, row4, row5, row6, row7, row8))
                conn.commit()
            except:
                pass
            
        